Build an ElectronicStructureProblem

In [6]:
'''PySCFDriver is a class that allows to build a molecluar object in PySCF. We use it to compute the one body and two body integrals.'''
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g", #slater-type orbitals approximated with 3 gaussians
    charge=0, #int chrge of the molecule
    spin=0, #2S where S spin of the molecule
    unit=DistanceUnit.ANGSTROM,
)

In [7]:
# Problem is a ElectronicStructureProblem object
problem = driver.run()
print(problem)

In [8]:
# The ElectonicStructureProblem object contains the hamiltonian of the molecule, and electronic integrals which the one and two body integrals
hamiltonian = problem.hamiltonian
coefficients = hamiltonian.electronic_integrals #one and two body integrals are 2d and 4d tensors respectively
print(coefficients.alpha)

Polynomial Tensor
 "+-":
[[-1.25633907e+00 -1.37083854e-17]
 [-6.07732712e-17 -4.71896007e-01]]
 "++--":
[[[[6.75710155e-01 1.69253442e-16]
   [1.56722377e-16 1.80931200e-01]]

  [[4.84650299e-17 1.80931200e-01]
   [6.64581730e-01 3.79897400e-16]]]


 [[[1.01440795e-16 6.64581730e-01]
   [1.80931200e-01 4.71502663e-17]]

  [[1.80931200e-01 3.78920172e-16]
   [6.59828421e-17 6.98573723e-01]]]]


In [9]:
#The hamiltonian object contains the second quantized operator (the two summands of the hamiltonian). 
#This is the electronic part of the hamiltonian (nuclear repulsion energy will be added later as a constant, is available as hamiltonian.nuclear_repulsion_energy)
fermionic_op = hamiltonian.second_q_op()
print(fermionic_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.09046559989211568 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211564 * ( +_0 +_1 -_0 -_1 )
+ 0.33229086512764827 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.09046559989211568 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211564 * ( +_0 +_3 -_2 -_1 )
+ 0.33229086512764827 * ( +_0 +_3 -_3 -_0 )
+ 0.3322908651276482 * ( +_1 +_0 -_0 -_1 )
+ 0.09046559989211574 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211565 * ( +_1 +_1 -_0 -_0 )
+ 0.3492868613660089 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276482 * ( +_1 +_2 -_2 -_1 )
+ 0.09046559989211574 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211565 * ( +_1 +_3 -_2 -_0 )
+ 0.3492868613660089 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.09046559989211568 * ( +_2 +_0 -_1 -_3 )
+ 0.0904655

Mapping the problem to qubit space

In [10]:
'''Jordan-Wigner transformation maps the fermionic operators to unitary operators acting on qubits (https://arxiv.org/abs/2110.12792).
Doing so, we can map the n-body fermionic operator to a circuit of n-qubit gates.
'''
'''EDIT: bug in qiskit_nature.second_q.transforms.qubit_mapper.py called by JordanWignerMapper. Calls to_do method on FermionicOp object, 
which is not implemented, as the note says. Another mapper should be used, e.g. ParityMapper'''

from qiskit_nature.second_q.mappers import ParityMapper
mapper = ParityMapper()

In [11]:
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)

-0.8105479805373275 * IIII
+ 0.1721839326191556 * IIIZ
- 0.2257534922240239 * IIZZ
+ 0.17218393261915554 * IZZI
- 0.2257534922240239 * ZZII
+ 0.12091263261776629 * IIZI
+ 0.16892753870087907 * IZZZ
+ 0.04523279994605784 * ZXIX
- 0.04523279994605784 * IXZX
- 0.04523279994605784 * ZXZX
+ 0.04523279994605784 * IXIX
+ 0.16614543256382414 * ZZIZ
+ 0.16614543256382414 * IZIZ
+ 0.17464343068300445 * ZZZZ
+ 0.12091263261776629 * ZIZI


/home/mattia-robbiano/.local/lib/python3.11/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


Classical Solver

In [12]:
'''ow we need to define a solver, an algorithm that will find the ground state of the hamiltonian. 
One example is the NumPyEigensolver, which uses the NumPy sparse eigenvalue solver to compute the eigenvalues. This is a classical algorithm'''
from qiskit.algorithms.eigensolvers import NumPyEigensolver

numpy_solver = NumPyEigensolver(k=1)
numpy_result = numpy_solver.compute_eigenvalues(qubit_jw_op)
print(numpy_result)

{   'aux_operators_evaluated': None,
    'eigenstates': [   Statevector([-3.19509634e-16-6.73210927e-19j,
             -1.17289053e-16-3.09563315e-17j,
              2.34236345e-17-5.04875540e-17j,
             -9.81323962e-01+1.56725926e-01j,
              5.54722592e-17-4.78331052e-17j,
              4.99073782e-16+6.99084327e-16j,
              1.10140120e-01-1.75903299e-02j,
             -8.00902758e-17+4.61589082e-17j,
             -1.52186641e-16-1.91987207e-17j,
              7.54150601e-18+1.80062104e-16j,
             -2.49984341e-16+1.50880401e-16j,
             -2.83140502e-17+8.07828726e-18j,
             -1.12463563e-16+5.45414416e-17j,
              4.61862206e-18-6.54292931e-17j,
             -1.72914826e-16+2.29780821e-16j,
             -1.46448028e-16+3.16164053e-17j],
            dims=(2, 2, 2, 2))],
    'eigenvalues': array([-1.85727503])}


VQE eigensolver

In [13]:
'''The 3 foundamental parts of VQE are:
1) Ansatz: a parametrized quantum circuit that is used to prepare the trial state. 
Here we use the UCCSD ansatz: https://arxiv.org/abs/2109.15176.
2) Estimator: a class that computes the expectation value of the hamiltonian on the trial state.
3) Optimizer: a classical algorithm that finds the parameters of the ansatz that minimize the expectation 
value of the hamiltonian 
'''
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    problem.num_spatial_orbitals,       
    problem.num_particles,              #number of electrons
    mapper,                             #same mapper used before (parity mapper)
    initial_state=HartreeFock(          #initial state is the Hartree-Fock state so that the ansatz is not too far from the ground state.
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [15]:
'''GroundStateEigensolver is a class that combines the mapper, the solver and the problem 
to solve the problem and return the result'''
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030144
  - computed part:      -1.857275030144
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035695
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948821583]
    - computed part:      [0.0  0.0  1.388948821583]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000121583]  Total: 0.000000121583
                 (debye): [0.0  0.0  -0.000000309033]  Total: 0.000000309033
 
